In [1]:
import pickle 
import pandas

In [2]:
benchmarks = pickle.load(open("results/results.pickle", "rb"))

In [3]:
algos = ["umapcuml", "umaplearn", "umapgpumap"]
types = ["unsupervised", "supervised"]

In [4]:
datasets = list(benchmarks.keys())

In [5]:
len(datasets)

7

In [6]:
datasets

['digits', 'fashion_mnist', 'cifar100', 'coil20', 'shuttle', 'mnist', 'scrna']

In [7]:
benchmarks

{'digits': {'umapcuml': [{'unsupervised': {'time': 0.4452664852142334,
     'trust': 0.9876636433389232},
    'supervised': {'time': 0.47392821311950684, 'trust': 0.9879378905549919},
    'xform': {'time': 0.15683841705322266, 'trust': 0.9847805133487961}},
   {'unsupervised': {'time': 0.18512916564941406, 'trust': 0.9874644094493787},
    'supervised': {'time': 0.2050032615661621, 'trust': 0.9864168160345743},
    'xform': {'time': 0.42592787742614746, 'trust': 0.9856609318173406}},
   {'unsupervised': {'time': 0.44254136085510254, 'trust': 0.9869448789027268},
    'supervised': {'time': 0.4717979431152344, 'trust': 0.9879864913274599},
    'xform': {'time': 0.14838576316833496, 'trust': 0.9858305953573715}},
   {'unsupervised': {'time': 0.36017560958862305, 'trust': 0.9863258882554476},
    'supervised': {'time': 0.474459171295166, 'trust': 0.9875988980757733},
    'xform': {'time': 0.15300321578979492, 'trust': 0.9862826829215756}}],
  'umaplearn': [{'unsupervised': {'time': 9.26321

In [8]:
final_results = {}

for dataset in datasets:
    d = benchmarks[dataset]
    final_results[dataset] = {}
    for algo in algos:
        if algo in d:
            a = d[algo]
            summary = {}
            for bench in a:
                for t in types:
                    if t not in summary:
                        summary[t] = { "sum": 0, "count": 0, "sum_squared": 0, "trust": 0}
                    if t in bench:
                        time = bench[t]["time"]
                        summary[t]["sum"] += time
                        summary[t]["count"] += 1
                        summary[t]["sum_squared"] += time**2
                        summary[t]["trust"] = max(summary[t]["trust"], bench[t]["trust"])

                final_results[dataset][algo] = {}
                for t in types:
                    if summary[t]["count"] > 0:
                        mean = (summary[t]["sum"]) / summary[t]["count"]
                        var = ((summary[t]["sum_squared"]) / summary[t]["count"])  - (mean**2)
                        trust = summary[t]["trust"]
                    else:
                        mean = 0
                        var = 0
                        trust = 0
                    final_results[dataset][algo][t] = {"mean": mean, "var": var, "trust": trust}
                        
        else:
            print(algo + " not in " + dataset)

In [9]:
final_results

{'digits': {'umapcuml': {'unsupervised': {'mean': 0.35827815532684326,
    'var': 0.011162857575371277,
    'trust': 0.9876636433389232},
   'supervised': {'mean': 0.40629714727401733,
    'var': 0.013507401316314116,
    'trust': 0.9879864913274599}},
  'umaplearn': {'unsupervised': {'mean': 6.328544795513153,
    'var': 2.8975128262992627,
    'trust': 0.9879530939808027},
   'supervised': {'mean': 6.755777180194855,
    'var': 0.1106131444620786,
    'trust': 0.9876400330669284}},
  'umapgpumap': {'unsupervised': {'mean': 2.482882022857666,
    'var': 1.0582028882037378,
    'trust': 0.9558078864163978},
   'supervised': {'mean': 2.5530967116355896,
    'var': 0.10950217290617204,
    'trust': 0.9548070534694238}}},
 'fashion_mnist': {'umapcuml': {'unsupervised': {'mean': 0.45551514625549316,
    'var': 0.006011408943862762,
    'trust': 0.9773246413616931},
   'supervised': {'mean': 1.036961555480957,
    'var': 0.0002095378332569453,
    'trust': 0.9762932845375539}},
  'umaplearn

In [10]:
data = []
for dataset in datasets:
    for algo in algos:
        for t in types:
            if algo in final_results[dataset]:
                b = final_results[dataset][algo][t]
                data.append((dataset, algo, t, b["mean"], b["var"], b["trust"]))

In [11]:
import csv

with open('benchmark_results.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['dataset','impl', 'bench', 'mean', 'var', 'max_trust'])
    for row in data:
        csv_out.writerow(row)

In [12]:
data

[('digits',
  'umapcuml',
  'unsupervised',
  0.35827815532684326,
  0.011162857575371277,
  0.9876636433389232),
 ('digits',
  'umapcuml',
  'supervised',
  0.40629714727401733,
  0.013507401316314116,
  0.9879864913274599),
 ('digits',
  'umaplearn',
  'unsupervised',
  6.328544795513153,
  2.8975128262992627,
  0.9879530939808027),
 ('digits',
  'umaplearn',
  'supervised',
  6.755777180194855,
  0.1106131444620786,
  0.9876400330669284),
 ('digits',
  'umapgpumap',
  'unsupervised',
  2.482882022857666,
  1.0582028882037378,
  0.9558078864163978),
 ('digits',
  'umapgpumap',
  'supervised',
  2.5530967116355896,
  0.10950217290617204,
  0.9548070534694238),
 ('fashion_mnist',
  'umapcuml',
  'unsupervised',
  0.45551514625549316,
  0.006011408943862762,
  0.9773246413616931),
 ('fashion_mnist',
  'umapcuml',
  'supervised',
  1.036961555480957,
  0.0002095378332569453,
  0.9762932845375539),
 ('fashion_mnist',
  'umaplearn',
  'unsupervised',
  45.870298981666565,
  10.232404426096

In [13]:
scale_results_noknn = pickle.load(open("results/scale_results_precompute_knn.pickle", "rb"))["umapcuml"]
scale_results_knn = pickle.load(open("results/scale_results_with_knn.pickle", "rb"))["umapcuml"]

In [27]:
xaxis = sorted([int(a) for a in scale_results_knn.keys()])

In [31]:
knn_with = { int(k): v[0]["unsupervised"]["time"] for k,v in scale_results_knn.items() }

In [32]:
knn_without = { int(k): v[0]["unsupervised"]["time"] for k,v in scale_results_noknn.items() }

In [34]:
with open('googlenews_knn_and_without.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['n_samples', 'with_knn', 'without_knn'])
    for n_samples in xaxis:
        csv_out.writerow((n_samples, knn_with[n_samples], knn_without[n_samples]))